In [1]:
!pip install anthropic gradio plotly networkx

In [2]:
# detective_system.py

from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import gradio as gr
import plotly.graph_objects as go
import networkx as nx
from typing import Dict, Tuple
import torch
import numpy as np

class DetectiveSystem:
    """AI Detective System using Hugging Face models"""

    def __init__(self):
        print("Initializing Detective System...")
        self.setup_models()
        self.load_patterns()

    def setup_models(self):
        """Initialize required models"""
        try:
            # Zero-shot classification
            self.classifier = pipeline(
                "zero-shot-classification",
                model="facebook/bart-large-mnli"
            )

            # Sentiment analysis for urgency detection
            self.sentiment = pipeline(
                "sentiment-analysis",
                model="distilbert-base-uncased-finetuned-sst-2-english"
            )

            # Text generation
            self.generator = pipeline(
                "text-generation",
                model="gpt2",
                max_length=200
            )

            print("Models loaded successfully!")
        except Exception as e:
            print(f"Error loading models: {str(e)}")

    def load_patterns(self):
        """Load investigation patterns"""
        self.patterns = {
            'financial_crime': {
                'keywords': ['money', 'bank', 'transaction', 'account', 'transfer'],
                'steps': [
                    "Review Financial Records",
                    "Track Money Flow",
                    "Identify Accounts Involved",
                    "Interview Relevant Parties",
                    "Document Evidence"
                ]
            },
            'cybercrime': {
                'keywords': ['hack', 'data', 'system', 'network', 'access'],
                'steps': [
                    "Secure Systems",
                    "Analyze Logs",
                    "Track Data Access",
                    "Review Network Traffic",
                    "Document Digital Evidence"
                ]
            },
            'physical_crime': {
                'keywords': ['break-in', 'theft', 'physical', 'damage', 'entry'],
                'steps': [
                    "Secure Crime Scene",
                    "Collect Physical Evidence",
                    "Interview Witnesses",
                    "Review Security Footage",
                    "Document Findings"
                ]
            }
        }

    def analyze_case(self, case_text: str) -> Tuple[str, go.Figure]:
        """Analyze case and create visualization"""
        try:
            # Detect case type
            case_type = self.detect_case_type(case_text)

            # Get urgency level
            urgency = self.get_urgency(case_text)

            # Generate analysis
            analysis = self.generate_analysis(case_text, case_type, urgency)

            # Create flow diagram
            flow_diagram = self.create_flow_diagram(case_type)

            return analysis, flow_diagram

        except Exception as e:
            return f"Error analyzing case: {str(e)}", None

    def detect_case_type(self, text: str) -> str:
        """Detect type of case"""
        result = self.classifier(
            text,
            candidate_labels=list(self.patterns.keys()),
            hypothesis_template="This is a {} case."
        )
        return result['labels'][0]

    def get_urgency(self, text: str) -> str:
        """Determine case urgency"""
        sentiment = self.sentiment(text)[0]
        score = sentiment['score']

        # Check for urgent keywords
        urgent_words = ['immediate', 'urgent', 'critical', 'emergency']
        has_urgent_words = any(word in text.lower() for word in urgent_words)

        if has_urgent_words or score > 0.8:
            return "HIGH"
        elif score > 0.5:
            return "MEDIUM"
        else:
            return "LOW"

    def generate_analysis(self, text: str, case_type: str, urgency: str) -> str:
        """Generate detailed analysis"""
        # Structure the analysis
        analysis = f"""## Case Analysis

### Case Type: {case_type.replace('_', ' ').title()}
### Urgency Level: {urgency}

### Key Points:
"""

        # Add relevant steps based on case type
        analysis += "\n### Investigation Steps:\n"
        for step in self.patterns[case_type]['steps']:
            analysis += f"- {step}\n"

        # Generate additional insights
        prompt = f"Based on the case: {text}\nKey findings:"
        insights = self.generator(prompt, max_length=150, num_return_sequences=1)[0]['generated_text']

        analysis += f"\n### Additional Insights:\n{insights}\n"

        # Add recommendations
        analysis += "\n### Recommendations:\n"
        recommendations = [
            "Document all findings thoroughly",
            "Maintain chain of custody for evidence",
            "Coordinate with relevant departments",
            "Prepare detailed reports"
        ]
        for rec in recommendations:
            analysis += f"- {rec}\n"

        return analysis

    def create_flow_diagram(self, case_type: str) -> go.Figure:
        """Create investigation flow diagram"""
        steps = self.patterns[case_type]['steps']

        # Create graph
        G = nx.DiGraph()

        # Add nodes and edges
        for i, step in enumerate(steps):
            G.add_node(step)
            if i > 0:
                G.add_edge(steps[i-1], step)

        # Create layout
        pos = nx.spring_layout(G)

        # Create edge trace
        edge_trace = go.Scatter(
            x=[], y=[],
            line=dict(width=2, color='#888'),
            hoverinfo='none',
            mode='lines'
        )

        for edge in G.edges():
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_trace['x'] += (x0, x1, None)
            edge_trace['y'] += (y0, y1, None)

        # Create node trace
        node_trace = go.Scatter(
            x=[], y=[],
            text=[],
            mode='markers+text',
            hoverinfo='text',
            marker=dict(
                size=20,
                color='lightblue',
                line=dict(width=2)
            ),
            textposition="top center"
        )

        for node in G.nodes():
            x, y = pos[node]
            node_trace['x'] += (x,)
            node_trace['y'] += (y,)
            node_trace['text'] += (node,)

        # Create figure
        fig = go.Figure(
            data=[edge_trace, node_trace],
            layout=go.Layout(
                title='Investigation Flow',
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                plot_bgcolor='white',
                paper_bgcolor='white',
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
            )
        )

        return fig

def create_interface():
    """Create Gradio interface"""
    detective = DetectiveSystem()

    with gr.Blocks(theme=gr.themes.Soft()) as interface:
        gr.Markdown(
            """
            # 🔍 AI Detective System
            ## Powered by Hugging Face Models
            Enter case details for instant analysis and investigation plan.
            """
        )

        with gr.Row():
            # Input Column
            with gr.Column():
                case_input = gr.Textbox(
                    label="Case Description",
                    placeholder="Describe the case with relevant details...",
                    lines=6
                )
                analyze_btn = gr.Button("Analyze Case", variant="primary")

            # Output Column
            with gr.Column():
                analysis_output = gr.Markdown(label="Analysis")
                flow_diagram = gr.Plot(label="Investigation Flow")

        # Example Cases
        gr.Examples(
            examples=[
                ["Bank reported unauthorized transactions totaling $50,000. Multiple transfers to offshore accounts between 2-3 AM. Security logs show login from unknown IP."],
                ["Office break-in reported at tech company. Servers accessed, sensitive data potentially stolen. Security cameras disabled during incident."],
                ["Employee reported suspicious email attachments. Multiple systems showing unusual network activity. Customer database may be compromised."]
            ],
            inputs=case_input,
            label="Example Cases"
        )

        # Handle analysis button click
        analyze_btn.click(
            detective.analyze_case,
            inputs=case_input,
            outputs=[analysis_output, flow_diagram]
        )

    return interface

# For Google Colab, run this:
if __name__ == "__main__":
    # Install required packages
    # !pip install transformers torch gradio plotly networkx

    # Create and launch interface
    interface = create_interface()
    interface.launch(share=True)

Initializing Detective System...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Models loaded successfully!
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d1d857564e38a4f8e9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
